In [ ]:
#==================================================================
#Program: patchSegmentation
#Version: 1.0
#Author: David Helminiak
#Date Created: 20 September 2024
#Date Last Modified: 4 October, 2024
#Description: Load WSI and visualize different patch segmentation methods/parameters
#Operation: Move back into main program directory before running.
#==================================================================

#Have the notebook fill more of the display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

#RNG seed value to ensure run-to-run consistency (-1 to disable)
manualSeedValue = 0

#Should warnings and info messages be shown during operation
debugMode = True

#Should progress bars be visualized with ascii characters
asciiFlag = False

#When splitting WSI images, what size should the resulting patches be (default: 400)
#Should remain consistent with patch sizes given for training
patchSize = 400

#What weight should be used when overlaying data
overlayWeight = 0.5

#What is the camera resolution in mm/pixel for the instrument that acquired the data being used
cameraResolution = 0.00454

#Load external libraries
exec(open("./CODE/EXTERNAL.py", encoding='utf-8').read())

#Colors for background and patches identified as foreground data
cmapPatch = colors.ListedColormap(['black', 'red'])

#Load/synchronize data labeling, drop excluded rows, and extract relevant metadata
def loadMetadata_WSI(filename):
    metadata = pd.read_csv(filename, header=0, names=['Sample', 'Label'], converters={'Sample':str, 'Label':str})
    return metadata['Sample']

def processWSI(filename):
    
    #Load WSI
    imageWSI = cv2.cvtColor(cv2.imread(filename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
    
    #Crop WSI to the foreground area using Otsu
    imageWSI_gray = cv2.cvtColor(imageWSI, cv2.COLOR_RGB2GRAY)
    x, y, w, h = cv2.boundingRect(cv2.threshold(imageWSI_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]) 
    imageWSI = imageWSI[y:y+h, x:x+w]
    imageWSI_gray = imageWSI_gray[y:y+h, x:x+w]

    #Pad the image as needed (as symmetrially as possible) for an even division by the specified patch size; compute numpatches per row/column
    padHeight = (int(np.ceil(imageWSI.shape[0]/patchSize))*patchSize)-imageWSI.shape[0]
    padWidth = (int(np.ceil(imageWSI.shape[1]/patchSize))*patchSize)-imageWSI.shape[1]
    padTop, padLeft = padHeight//2, padWidth//2
    padBottom, padRight = padTop+(padHeight%2), padLeft+(padWidth%2)
    imageWSI = np.pad(imageWSI, ((padTop, padBottom), (padLeft, padRight), (0, 0)))
    imageWSI_gray = np.pad(imageWSI_gray, ((padTop, padBottom), (padLeft, padRight)))
    numPatchesRow, numPatchesCol = imageWSI.shape[0]//patchSize, imageWSI.shape[1]//patchSize
    overlayBase = copy.deepcopy(imageWSI)

    #Split the WSI (color and grayscale) into patches and flatten
    imageWSI = imageWSI.reshape(numPatchesRow, patchSize, numPatchesCol, patchSize, imageWSI.shape[2]).swapaxes(1,2)
    imageWSI = imageWSI.reshape(-1, imageWSI.shape[2], imageWSI.shape[3], imageWSI.shape[4])
    imageWSI_gray = imageWSI_gray.reshape(numPatchseRow, patchSize, numPatchesCol, patchSize).swapaxes(1,2)
    imageWSI_gray = imageWSI_gray.reshape(-1, imageWSI_gray.shape[2], imageWSI_gray.shape[3])

    return imageWSI, imageWSI_gray, overlayBase, numPatchesRow, numPatchesCol
    
#Directory/file specification
dir_data = '.' + os.path.sep + 'DATA' + os.path.sep
dir_patches_data = dir_data + 'PATCHES' + os.path.sep
dir_patches_inputWSI = dir_patches_data + 'INPUT_WSI' + os.path.sep
file_patches_metadataWSI = dir_patches_inputWSI + 'metadata_WSI.csv'
dir_WSI_data = dir_data + 'WSI' + os.path.sep
dir_WSI_inputs = dir_WSI_data + 'INPUT_WSI' + os.path.sep
file_WSI_metadataWSI = dir_WSI_inputs + 'metadata_WSI.csv'

#Load metadata for WSI
sampleNames_patches = loadMetadata_WSI(file_patches_metadataWSI)
WSIFilenames_patches = np.asarray([dir_patches_inputWSI + sampleName + '.jpg' for sampleName in sampleNames_patches])
sampleNames_WSI = loadMetadata_WSI(file_WSI_metadataWSI)
WSIFilenames_WSI = np.asarray([dir_WSI_inputs + sampleName + '.jpg' for sampleName in sampleNames_WSI])
#sampleNames, WSIFilenames = sampleNames_patches, WSIFilenames_patches
#sampleNames, WSIFilenames = sampleNames_WSI, WSIFilenames_WSI
sampleNames, WSIFilenames = np.concatenate([sampleNames_patches, sampleNames_WSI]), np.concatenate([WSIFilenames_patches, WSIFilenames_WSI])

#Compute otsu threshold for the WSI
computeOtsu = False
if computeOtsu:
    otsuThresholds = []
    for filename in tqdm(WSIFilenames, desc='Otsu', leave=True, ascii=asciiFlag):
        imageWSI_gray = cv2.cvtColor(cv2.imread(filename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2GRAY)
        otsuThreshold = cv2.threshold(imageWSI_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
        otsuThresholds.append(otsuThreshold)
    otsuThresholds = np.asarray(otsuThresholds)

#Minimum value [0, 255] for a grayscale pixel to be considered as a foreground location during patch extraction
#Determined as minimum Otsu threshold across all available WSI
patchBackgroundValue = otsuThresholds.min()
print('Determined patchBackgroundValue:', patchBackgroundValue)

#print('Otsu - Min: '+str(otsuThresholds.min())+' Max: '+str(otsuThresholds.max())+' Mean: '+str(otsuThresholds.mean())+' Std: '+str(otsuThresholds.std()))

#Otsu threshold data for classifier-training WSI
#Otsu - Min: 21.0 Max: 64.0 Mean: 38.84848484848485 Std: 9.182068203416629

#Otsu threshold data for non-classifier-training WSI
#Otsu - Min: 11.0 Max: 65.0 Mean: 33.95813953488372 Std: 9.993165863755591

#Otsu threshold data for all WSI
#Otsu - Min: 11.0 Max: 65.0 Mean: 35.1067615658363 Std: 10.025376547631357


In [ ]:
#Specify a filename index
index = 3

#What is the minimum area/quantity (mm^2) of foreground data that should qualify a patch for classification
#Result should not exceed patchSize*cameraResolution
minimumForegroundArea = 1.0**2

#Load and process indexed WSI
print('Sample: ' + sampleNames[index])
imageWSI, imageWSI_gray, overlayBase, numPatchesRow, numPatchesCol = processWSI(WSIFilenames[index])

#Compute the ratio of pixel count to total area that should be required for a patch to be considered as potentially holding foreground data (1mm^2 occupied area)
#patchBackgroundRatio = ((minimumForegroundArea/cameraResolution)**2)/(patchSize**2)
patchBackgroundRatio = minimumForegroundArea/((patchSize*cameraResolution)**2)
if patchBackgroundRatio > 1.0: sys.exit('\nError - Minimum foreground area specified for foreground data exceeds the given patch size.')

#Determine patches that would be considered to hold foreground data
patchkMap = np.zeros(overlayBase.shape[:2])
patchIndex = 0
for rowNum in range(0, numPatchesRow):
    for colNum in range(0, numPatchesCol):
        if np.mean(imageWSI_gray[patchIndex] >= patchBackgroundValue) >= patchBackgroundRatio: 
            locationRow, locationColumn= rowNum*patchSize, colNum*patchSize
            patchMap[locationRow:locationRow+patchSize, locationColumn:locationColumn+patchSize] = 1
        patchIndex += 1

#Show locations of patches that would be extracted/classified
patchkMap = cmapPatch(patchMap)[:,:,:3].astype(np.uint8)*255
overlayMap = cv2.addWeighted(overlayBase, 1.0, patchMap, overlayWeight, 0.0)
plt.figure(figsize=(10,10))
plt.imshow(overlayMap)
plt.show()
plt.close()



In [ ]:
#For debugging/development
#Select, visualize, and evaluate a specific patch from the WSI

#Specify a row and column
rowNum, colNum = 0, 12

#Extract and visualize patch portion that exceeds the background threshold
locationRow, locationColumn= rowNum*patchSize, colNum*patchSize
overlayBase_gray = cv2.cvtColor(overlayBase, cv2.COLOR_RGB2GRAY)
patchImage = overlayBase_gray[locationRow:locationRow+patchSize, locationColumn:locationColumn+patchSize]
plt.figure(figsize=(5,5))
plt.imshow(patchImage >= patchBackgroundValue, cmap='gray')
plt.show()
plt.close()
valueMean = np.mean(patchImage >= patchBackgroundValue)
print('Foreground/Background Ratio', np.mean(valueMean))
if valueMean >= patchBackgroundRatio: print('Patch would be extracted')
else: print('Patch would not be extracted')